In [1]:
import boto.ec2
import boto3

In [2]:
%run aws_helper.py

In [3]:
si = SpotInstances()
ctq = ClusterTaskQueue()

In [4]:
sqs = boto3.resource('sqs')  

In [5]:
new_member_queue = sqs.get_queue_by_name(QueueName='cluster_new_member')
task_commitment_queue = sqs.get_queue_by_name(QueueName='cluster_task_commitment')
task_queue = sqs.get_queue_by_name(QueueName='cluster_task_queue')

In [6]:
#si.request(num_instances=50, subnet='b')

In [7]:
si.print_public_dns_names()

ec2-52-89-10-42.us-west-2.compute.amazonaws.com
ec2-52-88-240-67.us-west-2.compute.amazonaws.com
ec2-52-89-2-13.us-west-2.compute.amazonaws.com
ec2-52-89-11-170.us-west-2.compute.amazonaws.com
ec2-52-89-11-193.us-west-2.compute.amazonaws.com
ec2-52-89-7-152.us-west-2.compute.amazonaws.com
ec2-52-89-5-149.us-west-2.compute.amazonaws.com
ec2-52-26-26-62.us-west-2.compute.amazonaws.com
ec2-52-89-11-195.us-west-2.compute.amazonaws.com
ec2-52-89-11-32.us-west-2.compute.amazonaws.com
ec2-52-88-250-249.us-west-2.compute.amazonaws.com
ec2-52-88-254-166.us-west-2.compute.amazonaws.com
ec2-52-89-2-4.us-west-2.compute.amazonaws.com
ec2-52-88-254-121.us-west-2.compute.amazonaws.com
ec2-52-88-248-214.us-west-2.compute.amazonaws.com
ec2-52-89-2-24.us-west-2.compute.amazonaws.com
ec2-52-88-254-160.us-west-2.compute.amazonaws.com
ec2-52-88-243-239.us-west-2.compute.amazonaws.com
ec2-52-89-1-250.us-west-2.compute.amazonaws.com
ec2-52-89-11-189.us-west-2.compute.amazonaws.com
ec2-52-88-215-233.us-west-2

In [ ]:
ctq.add_task('hello yejllow 17:27')

In [262]:
ctq.add_task('git:pull')

In [285]:
ctq.add_task('parse:01and2:0:20')

In [263]:
ctq.add_task('parse:01and2')

In [80]:
run_id = '05'
total_ids = 20
for i in range(total_ids):
    ctq.add_task('parse:%s:%d:%d' % (run_id, i, total_ids))

In [264]:
ctq.add_task('quit')

In [284]:
msgs = new_member_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    msg.delete()

ec2-52-88-104-26.us-west-2.compute.amazonaws.com: new_member


In [279]:
msgs = task_commitment_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    msg.delete()

In [283]:
#Delete 'quit' message after the end
msgs = task_queue.receive_messages(MessageAttributeNames=['Worker'])
for msg in msgs:
    worker_dns_name = 'unknown'
    if msg.message_attributes is not None:
        worker_dns_name = msg.message_attributes.get('Worker')['StringValue']
    print('%s: %s' % (worker_dns_name, msg.body))
    #msg.delete()